In [1]:
import requests
import json
import numpy as np
import pandas as pd
import io
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [2]:
response_API = requests.get('https://api.mytimestation.com/v0.1/reports/?api_key=&id=34&Report_StartDate=2014-01-01&Report_EndDate=2014-01-31&exportformat=csv')
print(response_API.status_code)


200


In [159]:
data = response_API.text
len(data)

23567

In [160]:
df = pd.read_csv(io.StringIO(data), sep=',', header=0, on_bad_lines='warn', 
                 parse_dates={'Datetime': ['Date', 'Time']}, 
                 usecols=['Date', 'Name', 'Department', 'Device', 'Time', 'Activity', 'shift_id'], 
                 dtype={'Department': 'category', 'Activity': 'category', 'Device': 'category'})
df.head()

,Datetime,Name,Department,Device,Activity,shift_id
0,2014-01-01 10:28:00,Julie Brown,COFFEE,TimeStation-2,Punch In,shft_q02pymx2e8
1,2014-01-01 10:54:00,Matthew L Whitney,COFFEE,TimeStation-2,Punch In,shft_d0n5ym3lq0
2,2014-01-01 11:03:00,Frank Ladish,COFFEE,TimeStation-2,Punch In,shft_589o2noo18
3,2014-01-01 16:22:00,Julie Brown,COFFEE,TimeStation-2,Punch Out,shft_q02pymx2e8
4,2014-01-01 16:22:00,Frank Ladish,COFFEE,TimeStation-2,Punch Out,shft_589o2noo18


In [75]:
x = datetime(2016, 1, 1)
td = 0
t = 5
while t>0:
    print((x+timedelta(td)).date(), (x+timedelta(td+30)).date())
    td += 31
    t -= 1

2016-01-01 2016-01-31
2016-02-01 2016-03-02
2016-03-03 2016-04-02
2016-04-03 2016-05-03
2016-05-04 2016-06-03


In [161]:
url = 'https://api.mytimestation.com/v0.1/reports/'
api_key = '?api_key=e26p34m7nlxw20z8l591eyqvjrd8og3d'
report_id = '&id=34'
start = ''
end = ''
export_format = '&exportformat=csv'
data_remaining = True
d = datetime(2014, 2, 1)
td = 0
#df = pd.DataFrame()

while data_remaining == True:
    start = '&Report_StartDate='+str((d+timedelta(td)).date())
    end = '&Report_EndDate='+str((d+timedelta(td+30)).date())
    api_string = url+api_key+report_id+start+end+export_format    
    response_API = requests.get(api_string)
    data = response_API.text
    if len(data) > 1000:
        loop_df = pd.read_csv(io.StringIO(data), sep=',', header=0, on_bad_lines='warn', 
                     parse_dates={'Datetime': ['Date', 'Time']}, 
                     usecols=['Date', 'Name', 'Department', 'Device', 'Time', 'Activity', 'shift_id'], 
                     dtype={'Department': 'category', 'Activity': 'category', 'Device': 'category'})
        df = pd.concat([df, loop_df], ignore_index=True)
        td += 31
    else:
        data_remaining = False

In [162]:
df.dtypes

Datetime      datetime64[ns]
Name                  object
Department            object
Device                object
Activity            category
shift_id              object
dtype: object

In [142]:
df.Department.value_counts()

COFFEE    17432
BAKERY     4434
Daniel     1288
OWNER       526
Name: Department, dtype: int64

In [149]:
df['Department'] = df.Department.astype('category')

In [144]:
df.Device.value_counts()

TimeStation-6    7865
TimeStation-3    7015
TimeStation-8    2953
TimeStation-7    2420
TimeStation-5    1932
TimeStation-2     210
TimeStation-4     117
TimeStation-9       4
Name: Device, dtype: int64

In [150]:
df['Device'] = df.Device.astype('category')

In [153]:
df.dtypes

Datetime      datetime64[ns]
Name                  object
Department          category
Device              category
Activity            category
shift_id              object
dtype: object

In [168]:
print(df.Name.nunique())

67


In [169]:
df.to_csv('/Users/mymac/Desktop/ts_data.csv')